In [18]:
from utils.data_utils import split_dataset, TimeSeriesDataset
from utils.evaluation_utils import plot_multistep_forecast
#from utils.training_utils import train
import pandas as pd 
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [19]:
import torch.nn as nn
import torch
import os
import random
import numpy as np

def normal_loss(normal_dist, y):
    neg_log_likelihood = -normal_dist.log_prob(y)
    return torch.mean(neg_log_likelihood)


def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")


class DecompositionLayer(nn.Module):
    def __init__(self, kernel_size, n_features):
        super().__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=1, padding=0)
        self.n_features = n_features

    def forward(self, x):
        num_of_pads = (self.kernel_size - 1) // 2
        if self.kernel_size > self.n_features:
            front = x[:, 0:1, :].repeat(1, num_of_pads + 1, 1)
        else:
            front = x[:, 0:1, :].repeat(1, num_of_pads, 1)
        end = x[:, -1:, :].repeat(1, num_of_pads, 1)
        x_padded = torch.cat([front, x, end], dim=1)
        x_trend = self.avg(x_padded.permute(0, 2, 1)).permute(0, 2, 1)
        x_seasonal = x - x_trend
        return x_seasonal, x_trend


class ARNet(nn.Module):
    def __init__(
        self,
        p_lag: int,
        n_continous_features: int,
        n_categorial_features: int,
        future_steps: int,
        decomp_kernel_size: int = 7,
        batch_size: int = 8,
        model: str = "rlinear",
        optimization: str = "mse",
        modelling_task: str = "univariate",
        density: bool = False,
        depth:str = 'shallow'
    ):

        super(ARNet, self).__init__()
        self.model = model
        self.optimization = optimization
        self.n_categorial_features = n_categorial_features
        self.modelling_task = modelling_task
        self.density = density
        self.depth = depth

        if self.modelling_task == "univariate":
            print("Univatiate modelling")
            self.inflation_factor = 1
            print(f"inflation factor = {self.inflation_factor}")

        elif self.modelling_task == "multivariate":
            print("Multivariate modelling")
            self.inflation_factor = n_continous_features
            print(f"inflation factor = {self.inflation_factor}")

        else:
            raise NotImplementedError

        if self.optimization == "mse":
            self.criterion = nn.MSELoss()

        else:
            raise NotImplementedError

        if model == "dlinear":
            print("Dlinear activated")
            self.decomp_layer = DecompositionLayer(
                decomp_kernel_size, n_continous_features
            )
            if self.density:
                print("Density to be estimated")
                self.mu_trend_layer = nn.Linear(
                    p_lag * (n_continous_features + n_categorial_features), 1
                )
                self.mu_seasonal_layer = nn.Linear(
                    p_lag * (n_continous_features + n_categorial_features), 1
                )
                self.std_trend_layer = nn.Linear(
                    p_lag * (n_continous_features + n_categorial_features), 1
                )
                self.std_seasonal_layer = nn.Linear(
                    p_lag * (n_continous_features + n_categorial_features), 1
                )
            else:
                print("Points to be estimated")
                if depth == 'deep': 
                    print("With a deep network")
                    self.relu = nn.ReLU()
                    self.input_trend_1layer = nn.Linear(
                        p_lag * (n_continous_features + n_categorial_features),
                        p_lag * (n_continous_features + n_categorial_features),
                    )
                    self.input_seasonal_1layer = nn.Linear(
                        p_lag * (n_continous_features + n_categorial_features),
                        p_lag * (n_continous_features + n_categorial_features),
                    )
                    self.input_trend_2layer = nn.Linear(
                        p_lag * (n_continous_features + n_categorial_features),
                        p_lag * (n_continous_features + n_categorial_features),
                    )
                    self.input_seasonal_2layer = nn.Linear(
                        p_lag * (n_continous_features + n_categorial_features),
                        p_lag * (n_continous_features + n_categorial_features),
                    )
                    self.input_trend_3layer = nn.Linear(
                        p_lag * (n_continous_features + n_categorial_features),
                        p_lag * (n_continous_features + n_categorial_features),
                    )
                    self.input_seasonal_3layer = nn.Linear(
                        p_lag * (n_continous_features + n_categorial_features),
                        p_lag * (n_continous_features + n_categorial_features),
                    )
                    self.input_trend_4layer = nn.Linear(
                        p_lag * (n_continous_features + n_categorial_features),
                        future_steps * self.inflation_factor,
                    )
                    self.input_seasonal_4layer = nn.Linear(
                        p_lag * (n_continous_features + n_categorial_features),
                        future_steps * self.inflation_factor,
                    )
                elif depth == 'shallow': 
                    print("With a shallow network")
                    self.input_trend_layer = nn.Linear(
                        p_lag * (n_continous_features + n_categorial_features),
                        future_steps * self.inflation_factor,
                    )
                    self.input_seasonal_layer = nn.Linear(
                        p_lag * (n_continous_features + n_categorial_features),
                        future_steps * self.inflation_factor,
                    )
                else: 
                    raise NotImplementedError

        elif model == "rlinear":
            print("Rlinear activated")
            # if self.density:
            #    print('Density to be estimated')
            #    self.mu_layer = nn.Linear(p_lag * (n_continous_features + n_categorial_features), 1)
            #    self.std_layer = nn.Linear(p_lag * (n_continous_features + n_categorial_features), 1)
            # else:
            print("Points to be estimated")
            self.input_layer = nn.Linear(
                p_lag * (n_continous_features + n_categorial_features),
                future_steps * self.inflation_factor,
            )

        elif model == "rmlp":
            print("RMLP activated")
            self.input_layer = nn.Linear(
                p_lag * (n_continous_features + n_categorial_features),
                p_lag * (n_continous_features + n_categorial_features),
            )
            self.relu = nn.ReLU()
            # if self.density:
            #    print('Density to be estimated')
            #    self.mu_layer = nn.Linear(p_lag * (n_continous_features + n_categorial_features), 1)
            #    self.std_layer = nn.Linear(p_lag * (n_continous_features + n_categorial_features), 1)
            # else:
            print("Points to be estimated")
            self.output_layer = nn.Linear(
                p_lag * (n_continous_features + n_categorial_features),
                future_steps * self.inflation_factor,
            )

        else:
            raise NotImplementedError

        self.criterion = nn.MSELoss()
        self.dropout = nn.Dropout(p=0.2)
        self.p_lag = p_lag
        self.batch_size = batch_size
        self.n_continous_features = n_continous_features
        self.future_steps = future_steps

    def forward(self, input):
        input = input.float()
        new_input = input.reshape(
            self.batch_size,
            (self.n_continous_features + self.n_categorial_features),
            self.p_lag,
        )

        if self.model == "rlinear":
            continous_input = new_input[:, 0 : (self.n_continous_features), :]
            categorial_input = new_input[
                :,
                self.n_continous_features : (
                    self.n_continous_features + self.n_categorial_features
                ),
                :,
            ]

            # continous_input tranformation
            mean_values = torch.mean(continous_input, dim=2).reshape(
                self.batch_size, self.n_continous_features, 1
            )
            mean_adj_input = continous_input - mean_values
            std_values = torch.std(continous_input, dim=2).reshape(
                self.batch_size, self.n_continous_features, 1
            )
            eps_values = torch.full((self.batch_size, self.n_continous_features, 1), 1)
            standardized_input = mean_adj_input / (std_values + eps_values)

            # put all parts together again
            standardized_input = torch.cat((standardized_input, categorial_input), 1)
            standardized_input = self.dropout(standardized_input)

            y_hat = self.input_layer(
                standardized_input.reshape(
                    self.batch_size,
                    self.p_lag
                    * (self.n_continous_features + self.n_categorial_features),
                )
            )

            if self.modelling_task == "univariate":
                rev_mean = mean_values.squeeze(2)[
                    :, self.n_continous_features - 1
                ].reshape(self.batch_size, 1)
                rev_std = std_values.squeeze(2)[
                    :, self.n_continous_features - 1
                ].reshape(self.batch_size, 1)
                rev_eps = torch.full((self.batch_size, 1), 1)
            elif self.modelling_task == "multivariate":
                rev_mean_l = []
                for tensor in mean_values.reshape(
                    self.batch_size, self.n_continous_features
                ):
                    [
                        rev_mean_l.append(
                            torch.full((self.future_steps, 1), i.item()).reshape(
                                self.future_steps
                            )
                        )
                        for i in tensor
                    ]
                rev_mean = torch.cat(rev_mean_l).reshape(
                    self.batch_size, self.n_continous_features * self.future_steps
                )
                rev_std_l = []
                for tensor in std_values.reshape(
                    self.batch_size, self.n_continous_features
                ):
                    [
                        rev_std_l.append(
                            torch.full((self.future_steps, 1), i.item()).reshape(
                                self.future_steps
                            )
                        )
                        for i in tensor
                    ]
                rev_std = torch.cat(rev_std_l).reshape(
                    self.batch_size, self.n_continous_features * self.future_steps
                )
                rev_eps = torch.full(
                    (self.batch_size, self.n_continous_features * self.future_steps), 1
                )
            else:
                NotImplementedError

            y_hat = y_hat * (rev_std + rev_eps) + rev_mean

        elif self.model == "dlinear":
            continous_input = new_input[:, 0 : (self.n_continous_features), :]
            categorial_input = new_input[
                :,
                self.n_continous_features : (
                    self.n_continous_features + self.n_categorial_features
                ),
                :,
            ]

            # continous_input tranformation
            input_season, input_trend = self.decomp_layer(continous_input)
            input_season = torch.cat((input_season, categorial_input), 1)
            input_trend = torch.cat((input_trend, categorial_input), 1)
            input_season = self.dropout(input_season)
            input_trend = self.dropout(input_trend)

            if self.density:
                mu_trend = self.mu_trend_layer(
                    input_trend.reshape(
                        self.batch_size,
                        self.p_lag
                        * (self.n_continous_features + self.n_categorial_features),
                    )
                )
                std_trend = self.std_trend_layer(
                    input_trend.reshape(
                        self.batch_size,
                        self.p_lag
                        * (self.n_continous_features + self.n_categorial_features),
                    )
                )
                mu_season = self.mu_seasonal_layer(
                    input_season.reshape(
                        self.batch_size,
                        self.p_lag
                        * (self.n_continous_features + self.n_categorial_features),
                    )
                )
                std_season = self.std_seasonal_layer(
                    input_season.reshape(
                        self.batch_size,
                        self.p_lag
                        * (self.n_continous_features + self.n_categorial_features),
                    )
                )
                self.sofplus = torch.nn.Softplus()
            else:
                if self.depth == 'deep': 
                    input_season_reshaped = input_season.reshape(
                            self.batch_size,
                            self.p_lag
                            * (self.n_continous_features + self.n_categorial_features),
                        )
                    
                    input_trend_reshaped = input_trend.reshape(
                            self.batch_size,
                            self.p_lag
                            * (self.n_continous_features + self.n_categorial_features),
                        )
                    y_hat_season = self.input_seasonal_1layer(
                        input_season_reshaped
                    )
                    y_hat_season = self.relu(self.input_seasonal_2layer(y_hat_season)) + input_season_reshaped
                    y_hat_season = self.relu(self.input_seasonal_3layer(y_hat_season)) + input_season_reshaped
                    y_hat_season = self.relu(self.input_seasonal_4layer(y_hat_season))

                    y_hat_trend = self.input_trend_1layer(
                        input_trend_reshaped
                    )
                    y_hat_trend = self.relu(self.input_trend_2layer(y_hat_trend)) + input_trend_reshaped
                    y_hat_trend = self.relu(self.input_trend_3layer(y_hat_trend)) + input_trend_reshaped
                    y_hat_trend = self.relu(self.input_trend_4layer(y_hat_trend))
                elif self.depth == 'shallow': 
                    y_hat_season = self.input_seasonal_layer(
                        input_season.reshape(
                            self.batch_size,
                            self.p_lag
                            * (self.n_continous_features + self.n_categorial_features),
                        )
                    )
                    y_hat_trend = self.input_trend_layer(
                        input_trend.reshape(
                            self.batch_size,
                            self.p_lag
                            * (self.n_continous_features + self.n_categorial_features),
                        )
                    )
                else: 
                    raise NotImplementedError
                y_hat = y_hat_season + y_hat_trend

        elif self.model == "rmlp":
            continous_input = new_input[:, 0 : (self.n_continous_features), :]
            categorial_input = new_input[
                :,
                self.n_continous_features : (
                    self.n_continous_features + self.n_categorial_features
                ),
                :,
            ]

            # continous_input tranformation
            mean_values = torch.mean(continous_input, dim=2).reshape(
                self.batch_size, self.n_continous_features, 1
            )
            mean_adj_input = continous_input - mean_values
            std_values = torch.std(continous_input, dim=2).reshape(
                self.batch_size, self.n_continous_features, 1
            )
            eps_values = torch.full((self.batch_size, self.n_continous_features, 1), 1)
            standardized_input = mean_adj_input / (std_values + eps_values)

            # put all parts together again
            standardized_input = torch.cat((standardized_input, categorial_input), 1)
            standardized_input = self.dropout(standardized_input)
            y_hat = self.input_layer(
                standardized_input.reshape(
                    self.batch_size,
                    self.p_lag
                    * (self.n_continous_features + self.n_categorial_features),
                )
            )

            # prediction part
            standardized_input = self.dropout(standardized_input)
            y_hat = self.relu(
                self.input_layer(
                    standardized_input.reshape(
                        self.batch_size,
                        self.p_lag
                        * (self.n_continous_features + self.n_categorial_features),
                    )
                )
            )
            y_hat = (
                y_hat.reshape(
                    self.batch_size,
                    (self.n_continous_features + self.n_categorial_features),
                    self.p_lag,
                )
                + standardized_input
            )
            y_hat = self.output_layer(
                y_hat.reshape(
                    self.batch_size,
                    self.p_lag
                    * (self.n_continous_features + self.n_categorial_features),
                )
            )

            if self.modelling_task == "univariate":
                rev_mean = mean_values.squeeze(2)[
                    :, self.n_continous_features - 1
                ].reshape(self.batch_size, 1)
                rev_std = std_values.squeeze(2)[
                    :, self.n_continous_features - 1
                ].reshape(self.batch_size, 1)
                rev_eps = torch.full((self.batch_size, 1), 1)
            elif self.modelling_task == "multivariate":
                rev_mean_l = []
                for tensor in mean_values.reshape(
                    self.batch_size, self.n_continous_features
                ):
                    [
                        rev_mean_l.append(
                            torch.full((self.future_steps, 1), i.item()).reshape(
                                self.future_steps
                            )
                        )
                        for i in tensor
                    ]
                rev_mean = torch.cat(rev_mean_l).reshape(
                    self.batch_size, self.n_continous_features * self.future_steps
                )
                rev_std_l = []
                for tensor in std_values.reshape(
                    self.batch_size, self.n_continous_features
                ):
                    [
                        rev_std_l.append(
                            torch.full((self.future_steps, 1), i.item()).reshape(
                                self.future_steps
                            )
                        )
                        for i in tensor
                    ]
                rev_std = torch.cat(rev_std_l).reshape(
                    self.batch_size, self.n_continous_features * self.future_steps
                )
                rev_eps = torch.full(
                    (self.batch_size, self.n_continous_features * self.future_steps), 1
                )
            else:
                NotImplementedError

            y_hat = y_hat * (rev_std + rev_eps) + rev_mean

        else:
            raise NotImplementedError

        if self.density and self.model == "dlinear":
            normal_object = torch.distributions.normal.Normal(
                (mu_trend + mu_season),
                (self.sofplus(std_trend) + self.sofplus(std_season)),
            )
            return normal_object
        else:
            return y_hat


class Gating(nn.Module):
    def __init__(self, input_dim,
                 num_experts, dropout_rate=0.1):
        super(Gating, self).__init__()

        # Layers
        self.layer1 = nn.Linear(input_dim, 128)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.layer2 = nn.Linear(128, 256)
        self.leaky_relu1 = nn.LeakyReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        self.layer3 = nn.Linear(256, 128)
        self.leaky_relu2 = nn.LeakyReLU()
        self.dropout3 = nn.Dropout(dropout_rate)

        self.layer4 = nn.Linear(128, num_experts)

    def forward(self, x):
        x = x.float()
        x = torch.relu(self.layer1(x))
        x = self.dropout1(x)

        x = self.layer2(x)
        x = self.leaky_relu1(x)
        x = self.dropout2(x)

        x = self.layer3(x)
        x = self.leaky_relu2(x)
        x = self.dropout3(x)

        return torch.softmax(self.layer4(x), dim=1)


class MoE(nn.Module):
    def __init__(self, trained_experts:list[ARNet]):
        super(MoE, self).__init__()
        self.experts = nn.ModuleList(trained_experts)
        self.criterion = nn.MSELoss()

        for expert in self.experts:
            for param in expert.parameters():
                param.requires_grad = False

        num_experts = len(trained_experts)
        input_dim = self.experts[0].p_lag * (self.experts[0].n_continous_features + self.experts[0].n_categorial_features)
        self.gating = Gating(input_dim, num_experts)

    def forward(self, x):
        #print('weights')
        weights = self.gating(x)
        #print(weights)
        #print('----------------------------')
        #print('outputs')
        outputs = torch.stack(
            [expert(x) for expert in self.experts], dim=2)
        print(outputs.shape)
        weights = torch.reshape(weights, outputs.shape)
        print(weights.shape)
        #weights = weights.unsqueeze(1).expand_as(outputs)
        return torch.sum(outputs * weights, dim=2)

In [20]:
import torch
from utils.model_utils import ARNet, set_seed, normal_loss, MoE
from torch.utils.data import DataLoader
from utils.data_utils import TimeSeriesDataset
from utils.metrics import metric
import random

def train_expert_or_moe(
        net,
        learning_rate: float, 
        train_data:DataLoader, 
        modelling_task: str, 
        n_continous_features:int, 
        batch_size:int, 
        val_loss_list:list, 
        val_data:DataLoader, 
        train_loss_list:list, 
        i:int, 
        density: bool = False, 
        num_of_experts: int = 2, 
        epochs:int = 2,
        moe: bool = False): 
    
    if moe == False: 
        print(f'Started training expert {i +1}/{num_of_experts}')
    else: 
        print('Started training Mixture of Experts')
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    for epoch in range(epochs):
        train_counter = 0
        val_counter = 0
        running_train_loss = 0.0
        running_val_loss = 0.0
        running_train_mae = 0.0
        running_train_mse = 0.0
        running_val_mae = 0.0
        running_val_mse = 0.0

        if epoch + 1 != 1 and (epoch + 1) % 2 == 0:
            learning_rate = learning_rate / 2
            optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
        print(f"Current learning rate is : {learning_rate}")
        print("---------------------------")
        for i, data in enumerate(train_data):
            inputs, labels = data
            labels = labels.squeeze(0).float()

            optimizer.zero_grad()
            outputs = net(inputs)
            if density:
                loss = normal_loss(outputs, labels.squeeze(1))
            else:
                if modelling_task == "multivariate":
                    loss = net.criterion(
                        outputs,
                        labels[:, 0:(n_continous_features), :].reshape(outputs.shape),
                        )
                else:
                    loss = net.criterion(outputs, labels.squeeze(1))

            loss.backward()
            # torch.nn.utils.clip_grad_norm_(net.parameters(), 1)
            optimizer.step()
            if density == False:
                outputs_array = outputs.detach().cpu().numpy()
                labels_array = labels.squeeze(2).detach().cpu().numpy()
                mae, mse = metric(pred=outputs_array, true=labels_array)
                running_train_mae += mae
                running_train_mse += mse

            running_train_loss += loss.item()
            train_counter += batch_size
            if train_counter % 5000 == 0:
                print(
                    f"Current (running) training loss at iteration {train_counter} : {running_train_loss/train_counter}"
                    )
        train_loss_list.append(running_train_loss / train_counter)

        if density == False:
            for i, data in enumerate(val_data):
                inputs, test_labels = data
                test_labels = test_labels.squeeze(0).float()
                output = net(inputs)
                if modelling_task == "multivariate":
                    val_loss = net.criterion(
                        outputs,
                        test_labels[:, 0:(n_continous_features), :].reshape(
                            outputs.shape
                            ),
                        )
                else:
                    val_loss = net.criterion(output, test_labels.squeeze(1))

                running_val_loss += val_loss.item()

                output_array = output.detach().cpu().numpy()
                test_labels_array = test_labels.squeeze(2).detach().cpu().numpy()
                mae, mse = metric(pred=output_array, true=test_labels_array)
                running_val_mae += mae
                running_val_mse += mse
                val_counter += batch_size
            val_loss_list.append(running_val_loss / val_counter)

            if epoch % 1 == 0:
                print("")
                print(f"Epoch {epoch}: ")
                print("")
                print("Train metrics: -------")
                print(f"Running (training) loss is {running_train_loss/train_counter}.")
                print(f"Training MAE is {running_train_mae/train_counter}.")
                print(f"Training MSE is {running_train_mse/train_counter}.")
                print("")
                print("Test metrics: -------")
                print(f"Running (test) loss is {running_val_loss/val_counter}.")
                print(f"Test MAE is {running_val_mae/val_counter}.")
                print(f"Test MSE is {running_val_mse/val_counter}.")
                print("---------------------------")
    return net

def train(
    epochs,
    p_lag,
    future_steps,
    n_continous_features,
    n_categorial_features,
    training_df,
    validation_df,
    feature_columns,
    dataset_name: str,
    target_column=["OT"],
    learning_rate=1.0e-4,
    decomp_kernel_size=7,
    batch_size=8,
    model="rlinear",
    moe: bool = False, 
    num_of_experts: int = 1, 
    modelling_task="univariate",
    density=False,
    depth = 'shallow'
):

    untrained_experts = []
    for _ in range(num_of_experts): 
        set_seed(random.randint(3, 1000))
        net = ARNet(
                p_lag=p_lag,
                n_continous_features=n_continous_features,
                n_categorial_features=n_categorial_features,
                future_steps=future_steps,
                decomp_kernel_size=decomp_kernel_size,
                batch_size=batch_size,
                model=model,
                modelling_task=modelling_task,
                density=density,
                depth = depth
            )
        untrained_experts.append(net)

    train_data = DataLoader(
        TimeSeriesDataset(
            training_df,
            future_steps=future_steps,
            feature_columns=feature_columns,
            target_column=target_column,
            p_lag=p_lag,
            modelling_task=modelling_task,
        ),
        batch_size=batch_size,
        drop_last=True,
    )
    train_loss_list = []
    val_data = DataLoader(
        TimeSeriesDataset(
            validation_df,
            future_steps=future_steps,
            feature_columns=feature_columns,
            target_column=target_column,
            p_lag=p_lag,
            modelling_task=modelling_task,
        ),
        batch_size=batch_size,
        drop_last=True,
    )
    val_loss_list = []

    torch.set_grad_enabled(True)
    
    trained_experts = []
    for i in range(num_of_experts): 
        net = train_expert_or_moe(
                 learning_rate=learning_rate, 
                 num_of_experts=num_of_experts, 
                 net=untrained_experts[i], 
                 epochs=epochs, 
                 train_data=train_data, 
                 density=density, 
                 modelling_task=modelling_task, 
                 n_continous_features=n_continous_features, 
                 batch_size=batch_size, 
                 val_loss_list=val_loss_list, 
                 val_data=val_data, 
                 train_loss_list=train_loss_list, 
                 i=i, 
                 moe = False)
        trained_experts.append(net)
    if moe: 
        moe_model = MoE(trained_experts)
        moe_model = train_expert_or_moe(
            num_of_experts=num_of_experts, 
            learning_rate=learning_rate, 
            net=moe_model, 
            epochs=epochs, 
            train_data=train_data, 
            density=density, 
            modelling_task=modelling_task, 
            n_continous_features=n_continous_features, 
            batch_size=batch_size, 
            val_loss_list=val_loss_list, 
            val_data=val_data, 
            train_loss_list=train_loss_list, 
            i=0, 
            moe = True)
        return moe_model
    
    else: 
        return net


In [21]:
ETTm2 = pd.read_csv("/workspaces/time_series_experiment/ETT-small/ETTm2.csv")
training_df, test_df = split_dataset(ETTm2, remain_same = False)

In [22]:
p_lag = 48
future_steps = 24
batch_size = 8
epochs = 1
learning_rate=1.e-4
decomp_kernel_size = 24
number_of_forecasts = 100
target_column = ['OT']
feature_columns = [i for i in training_df.columns]
modelling_task = 'univariate'
n_continous_features=7
n_categorial_features=5
dataset_name = 'ETTm2LongTraining'
moe = True
number_of_experts = 2

In [23]:
net = train(
            epochs = epochs, 
            n_continous_features=n_continous_features, 
            n_categorial_features=n_categorial_features,
            p_lag=  p_lag, 
            future_steps = future_steps, 
            training_df = training_df, 
            validation_df = test_df, 
            feature_columns = feature_columns,
            target_column = target_column, 
            learning_rate=learning_rate ,
            decomp_kernel_size= decomp_kernel_size, 
            batch_size=batch_size, 
            model = 'rlinear', 
            modelling_task = modelling_task, 
            dataset_name = dataset_name, 
            moe = moe, 
            num_of_experts = number_of_experts
            )
test_data = DataLoader(TimeSeriesDataset(test_df, future_steps= future_steps, target_column = target_column,feature_columns=feature_columns,p_lag=p_lag), batch_size=batch_size,drop_last=True)
plot_multistep_forecast(test_data=test_data, dataset_name = dataset_name, neural_net=net, future_steps=future_steps, number_of_forecasts=number_of_forecasts)

Random seed set as 244
Univatiate modelling
inflation factor = 1
Rlinear activated
Points to be estimated
Random seed set as 971
Univatiate modelling
inflation factor = 1
Rlinear activated
Points to be estimated
Started training expert 1/2
Current learning rate is : 0.0001
---------------------------
Current (running) training loss at iteration 5000 : 16.95312412471771


KeyboardInterrupt: 